# Initial Setup

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
!pip install wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.4MB 4.8MB/s 
     |████████████████████████████████| 112kB 29.8MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 460kB 23.4MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=bd431a7c7f187272be19a9836b1e7616f86e03d34c0a4de4b4f6810140d9de42
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=c9fed8e42385522813f4c68267548bd110f3f869f0abc97b6fe982888c9aabf6
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=0d

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [0]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from wandb.keras import WandbCallback
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import time
import cv2
from tqdm.notebook import tqdm
from imutils import paths
tf.random.set_seed(666)
np.random.seed(666)

tfds.disable_progress_bar()

# Imagenet Subset 

In [4]:
!git clone https://github.com/thunderInfy/imagenet-5-categories


Cloning into 'imagenet-5-categories'...
remote: Enumerating objects: 1532, done.
remote: Total 1532 (delta 0), reused 0 (delta 0), pack-reused 1532
Receiving objects: 100% (1532/1532), 88.56 MiB | 51.26 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [0]:
# Train and test image paths
train_images = list(paths.list_images("imagenet-5-categories/train"))
test_images = list(paths.list_images("imagenet-5-categories/test"))


In [0]:
def prepare_images(image_paths):
    images = []
    labels = []

    for image in tqdm(image_paths):
        image_pixels = plt.imread(image)
        image_pixels = cv2.resize(image_pixels, (128,128))
        image_pixels = image_pixels/255.

        label = image.split("/")[2].split("_")[0]

        images.append(image_pixels)
        labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    print(images.shape, labels.shape)

    return images, labels

In [7]:
X_train, y_train = prepare_images(train_images)
X_test, y_test = prepare_images(test_images)


(1250, 128, 128, 3) (1250,)



(250, 128, 128, 3) (250,)


In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)


In [0]:
train_ds=tf.data.Dataset.from_tensor_slices((X_train,y_train_enc))
validation_ds=tf.data.Dataset.from_tensor_slices((X_test,y_test_enc))

In [0]:
@tf.function
def aug(image, label):
  x=tf.image.random_brightness(image,max_delta=0)
  x=tf.image.random_contrast(x,lower=0.2, upper=1.8)
  x = tf.image.random_saturation(x, lower=0.2, upper=1.5)
  x = tf.image.random_hue(x, max_delta=0.4)
  x = tf.clip_by_value(x, 0, 1)

  return x, label

In [0]:
IMG_SHAPE = 128
BS = 64
AUTO = tf.data.experimental.AUTOTUNE
train_ds = (
    train_ds
    .shuffle(100)
    .batch(BS)
    .map(aug, num_parallel_calls=AUTO)
    .prefetch(AUTO)
)
validation_ds = (
    validation_ds
    .shuffle(100)
    .batch(BS)
    .prefetch(AUTO)
)

# Model building and training wih Adam


In [0]:
resnet50 = tf.keras.applications.ResNet50(weights=None, include_top=False)
model = tf.keras.Sequential([resnet50,GlobalAveragePooling2D(),Dropout(0.25),Dense(5,activation='softmax')])

In [0]:
decay_steps = 1000
lr_decayed_fn = tf.keras.experimental.CosineDecay(
    initial_learning_rate=0.001, decay_steps=decay_steps)

model.compile(optimizer=tf.keras.optimizers.Adam(lr_decayed_fn),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [0]:
es = tf.keras.callbacks.EarlyStopping(monitor="val_sparse_categorical_accuracy", patience=2,
	restore_best_weights=True, verbose=2)

In [24]:
import time
import wandb

wandb.init(entity='authors',project='scl',id="ADA")
start = time.time()
model.fit(train_ds,
          validation_data=validation_ds,
          epochs=50,
          callbacks=[wandb.keras.WandbCallback(), es])
end = time.time()
wandb.log({"training_time": end - start})

Epoch 1/50
20/20 [==============================] - 5s 234ms/step - loss: 1.6424 - sparse_categorical_accuracy: 0.2544 - val_loss: 1.6301 - val_sparse_categorical_accuracy: 0.2480
Epoch 2/50
20/20 [==============================] - 3s 143ms/step - loss: 1.5880 - sparse_categorical_accuracy: 0.3136 - val_loss: 1.7048 - val_sparse_categorical_accuracy: 0.2000
Epoch 3/50
20/20 [==============================] - 3s 145ms/step - loss: 1.5839 - sparse_categorical_accuracy: 0.3200 - val_loss: 1.7048 - val_sparse_categorical_accuracy: 0.2000
Epoch 00003: early stopping


In [0]:
model.save_weights("full_supervised_learning.h5")

In [0]:
wandb.save("full_supervised_learning.h5")

['/content/wandb/run-20200528_111108-2h40mbhd/full_supervised_learning.h5']